In [1]:
import pandas as pd
import requests
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from pathlib import Path
import os

/Users/Rebeca/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Rebeca/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Rebeca/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Rebeca/opt/anaconda3/lib/python3.7/site-packages/tensorflow/

In [2]:
from collections import Counter

In [2]:
PTT_stock = pd.read_csv('/Users/Rebeca/Desktop/Fintech/PPT_Stock.csv')
PTT_stock.head()

,site,title,author,date,content
0,PTT-STOCK 版,[新聞] 營收：訊芯-KY(6451)4月營收3億9035萬元,scum5566 (你好宅),Fri May 8 17:31:43 2020,\n\n1.原文連結：\n\n\nhttps://tw.stock.yahoo.com/ne...
1,PTT-STOCK 版,[新聞] 聯上發（2537）四月營收報告,cpblgu (《善良溫柔》的心愛),Fri May 8 17:06:15 2020,\n【財訊快報／編輯部】聯上發(2537)自結109年4月營收5億9996萬5000元，和上...
2,PTT-STOCK 版,[新聞] 財報：遠見(3040)109年第一季淨利歸屬母,j5163124 (Little Y),Fri May 8 16:49:06 2020,\n-發文提醒\n1.發文前請先詳閱[新聞]分類發文規範，未依規範發文將受處份。\n2.連結...
3,PTT-STOCK 版,[新聞] 寶齡富錦3月稅後虧損700萬元 每股虧0.09,tonyparker18 (tonyparker183),Fri May 8 16:30:37 2020,\n\n1.原文連結：\n\nhttps://is.gd/5LYmBd\n\n2.原文內容：...
4,PTT-STOCK 版,[新聞] 半導體 Switch滑鼠撐腰原相Q2營收拚季增5,kria5304 (XenoMegaREENovaSaga),Fri May 8 16:12:33 2020,\n\n1.原文連結：\nhttps://tinyurl.com/y7633pvl\n\n2...


In [15]:
# 內文存成一個list
contents=[]
for i in range(len(PTT_stock['content'])):
    contents.append(PTT_stock['content'][i])

# 匯入股票資料

In [6]:
stock_df = pd.read_csv('/Users/Rebeca/Fintech_中信專案/stock_list.csv')
stock_df.head()

,Unnamed: 0,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode,備註
0,2,1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR,NaN
1,3,1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR,NaN
2,4,1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR,NaN
3,5,1104 環泥,TW0001104008,1971/02/01,上市,水泥工業,ESVUFR,NaN
4,6,1108 幸福,TW0001108009,1990/06/06,上市,水泥工業,ESVUFR,NaN


In [7]:
# 將股票名稱、代號放入list
stock_num=[]
stock_name=[]
for i in stock_df['有價證券代號及名稱']:
    try:
        stock_num.append(i.split("\u3000")[0])
        stock_name.append(i.split("\u3000")[1])
    except:
        print(i)

4148 全宇生技-KY
上市認購(售)權證
ETN
特別股
ETF
ETN
臺灣存託憑證(TDR)
受益證券-不動產投資信託


# 斷詞

## CKIP

In [10]:
path = str(Path.home())+'/ckip/'
if not os.path.exists(path):
    os.mkdir(path)
    data_utils.download_data_gdown(path)

In [11]:
ws = WS(path+"/data")
pos= POS(path+"/data")
ner= NER(path+"/data")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### 自定義辭典

In [12]:
user_dic={}
for i in stock_num:
    dic3={i:1}
    user_dic.update(dic3)
for i in stock_name:
    dic3={i:1}
    user_dic.update(dic3)
len(user_dic) #作為word_to_weight

33329

In [13]:
word_to_weight = user_dic
dictionary = construct_dictionary(word_to_weight)

### Run the WS-POS-NER pipeline

In [16]:
word_sentence_list = ws(
    contents,
    recommend_dictionary = dictionary
)

pos_sentence_list = pos(word_sentence_list)

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

### Show Results

In [19]:
word_sentence_list

[['\n\n1',
  '.',
  '原文',
  '連結',
  '：\n\n\nhttps://tw.stock.yahoo.com/news/%E7%',
  '87%',
  '9',
  'F%E6%',
  '94%',
  'B',
  '6-%E8%A8%8A%E8%8A%AF-ky-6451-',
  '4%E6%',
  '9C%',
  '88%',
  'E7%',
  '87%',
  '9F%E6%',
  '94%',
  'B',
  '63%',
  'E',
  '5%',
  '84%',
  '849035%',
  'E',
  '8%',
  '90%',
  'AC%E5%',
  '85%',
  '83',
  '-',
  '084355649',
  '.',
  'html\nhttps://tinyurl',
  '.',
  'com/ybcgqwwv\n\n2',
  '.',
  '原文',
  '內容',
  '：',
  '\n\n',
  '營收',
  '：',
  '訊芯-KY',
  '(6451)',
  '4月',
  '營收',
  '3億9035萬',
  '元',
  '，',
  '月增率',
  '-',
  '1.38%',
  '，',
  '年增率',
  '-',
  '26.39%',
  '\n\n',
  '【',
  '財訊',
  '快報',
  '／',
  '編輯部',
  '】',
  '訊芯-KY',
  '(6451)',
  '自結',
  '109年',
  '4月',
  '營收',
  '3億9035萬',
  '3000',
  '元',
  '，',
  '和',
  '上',
  '個',
  '月',
  '的',
  '3\n億9583萬7000',
  '元',
  '相較',
  '，',
  '減少',
  '金額',
  '為',
  '548萬4000',
  '元',
  '，',
  '較',
  '上',
  '月',
  '減少',
  '1.38%',
  '，',
  '和',
  '108年',
  '同',
  '月',
  '的',
  '5億\n3032萬',
  '元',
  '相較',
  '，

In [18]:
entity_sentence_list


[{(219, 221, 'DATE', '4月'),
  (223, 231, 'MONEY', '3億9035萬元'),
  (236, 241, 'PERCENT', '1.38%'),
  (285, 297, 'MONEY', '3億9035萬3000元'),
  (306, 316, 'MONEY', '9583萬7000元'),
  (324, 333, 'MONEY', '548萬4000元'),
  (335, 337, 'DATE', '上月'),
  (339, 344, 'PERCENT', '1.38%'),
  (346, 352, 'DATE', '108年同月'),
  (356, 362, 'MONEY', '3032萬元'),
  (370, 382, 'MONEY', '1億3996萬7000元'),
  (384, 390, 'DATE', '108年同月'),
  (392, 398, 'PERCENT', '26.39%'),
  (403, 411, 'DATE', '109年1-4月'),
  (413, 426, 'MONEY', '13億9585萬1000元'),
  (428, 432, 'DATE', '108年'),
  (435, 448, 'MONEY', '19億5714萬2000元'),
  (457, 469, 'MONEY', '5億6129萬1000元'),
  (471, 475, 'DATE', '108年'),
  (479, 485, 'PERCENT', '28.68%'),
  (541, 544, 'DATE', '星期一')},
 {(2, 6, 'ORG', '財訊快報'),
  (11, 14, 'ORG', '聯上發'),
  (22, 28, 'DATE', '109年4月'),
  (30, 42, 'MONEY', '5億9996萬5000元'),
  (54, 62, 'MONEY', '75萬5000元'),
  (70, 78, 'MONEY', '3億2221萬元'),
  (80, 82, 'DATE', '上月'),
  (84, 91, 'PERCENT', '116.00%'),
  (93, 99, 'DATE', '108年同月'),
  (110

In [20]:
len(word_sentence_list)==len(entity_sentence_list)

True

### 將股票代碼、名稱與NER整理成每一篇一個list

In [21]:
lineNames = []  

for i, sentence in enumerate(word_sentence_list):
    lineNames.append([]) 
    for entity in sorted(entity_sentence_list[i]):
            lineNames[-1].append(entity[3])
    for word in sentence:
        if (word in stock_num) or (word in stock_name) and (word not in lineNames[-1]):
            lineNames[-1].append(word)     

In [22]:
len(lineNames)

220

### 將股票代碼、名稱當做key

In [29]:
stock_node={}
for words in lineNames:
    for w in words:
        if (w in stock_num) or (w in stock_name):
            words.remove(w)
            if stock_node.get(w) is None:
                stock_node[w] = words
            else:
                stock_node[w] = stock_node[w]+words
                

In [30]:
len(stock_node)

65

In [31]:
stock_node.keys()

dict_keys(['大量', '旺宏', '2352', '2337', '2020', '2024', '大立光', '旭富', '巨大', '台塑', '元大S&P石油', 'ETF', '國巨', '2454', '新光金', '三星', '2448', '惠特', '工信', '友達', '鴻海', '聯詠', '友訊', '明泰', '佳世達', '羅昇', '宏達電', '2017', '全國', '國光生', '元大台灣50', '瑞昱', '聯電', '寶成', '中鋼', '中租-KY', '大億', '新興', '力成', '6116', '2002', '台積電', '1402', '無敵', '群創', '元大金', '中化', '禾伸堂', '台新金', '聯發科', '玉山金', '華新科', '華南金', '太空梭', '豐泰', '冠軍', '光鋐', '廣達', '和碩', '統一超', '大聯大', '中華', '第一金', '長榮航', '合庫金'])

### 將股票代碼、名稱轉成df，後用做node資料

In [36]:
stock_key=[]
for i in stock_node:
    stock_key.append(i)

In [37]:
id=[]
for i in range(len(stock_key)):
    id.append(i)

In [44]:
node.head()

,id,lable
0,0,大量
1,1,旺宏
2,2,2352
3,3,2337
4,4,2020


In [71]:
id

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64]

In [72]:
stock_key

['大量',
 '旺宏',
 '2352',
 '2337',
 '2020',
 '2024',
 '大立光',
 '旭富',
 '巨大',
 '台塑',
 '元大S&P石油',
 'ETF',
 '國巨',
 '2454',
 '新光金',
 '三星',
 '2448',
 '惠特',
 '工信',
 '友達',
 '鴻海',
 '聯詠',
 '友訊',
 '明泰',
 '佳世達',
 '羅昇',
 '宏達電',
 '2017',
 '全國',
 '國光生',
 '元大台灣50',
 '瑞昱',
 '聯電',
 '寶成',
 '中鋼',
 '中租-KY',
 '大億',
 '新興',
 '力成',
 '6116',
 '2002',
 '台積電',
 '1402',
 '無敵',
 '群創',
 '元大金',
 '中化',
 '禾伸堂',
 '台新金',
 '聯發科',
 '玉山金',
 '華新科',
 '華南金',
 '太空梭',
 '豐泰',
 '冠軍',
 '光鋐',
 '廣達',
 '和碩',
 '統一超',
 '大聯大',
 '中華',
 '第一金',
 '長榮航',
 '合庫金']

In [73]:
lable=[]
for i, nodes in enumerate(stock_node):
    for j,value in enumerate(stock_node[nodes]):
        i = str(i)
        j = str(j)
        num = (i,j)
        num2 = "-".join(num)
        id.append(num2)
        stock_key.append(value)

In [79]:
color=[]
for i in range(len(id)):
    color.append('#ffea00')

In [80]:
node_dic={'id':id,'lable':stock_key,'color':color}
node = pd.DataFrame(node_dic)

In [81]:
node

,id,lable,color
0,0,大量,#ffea00
1,1,旺宏,#ffea00
2,2,2352,#ffea00
3,3,2337,#ffea00
4,4,2020,#ffea00
5,5,2024,#ffea00
6,6,大立光,#ffea00
7,7,旭富,#ffea00
8,8,巨大,#ffea00
9,9,台塑,#ffea00


In [82]:
node.to_csv('/Users/Rebeca/Fintech_中信專案/node.csv')

In [87]:
strat=[]
to=[]
for i in id[65:]:
    f = i.split('-')[0]
    strat.append(f)
    to.append(i)

In [89]:
edges_dic = {'from':strat,'to':to}
edges = pd.DataFrame(edges_dic) 

In [90]:
edges

,from,to
0,0,0-0
1,0,0-1
2,0,0-2
3,0,0-3
4,0,0-4
5,0,0-5
6,0,0-6
7,0,0-7
8,0,0-8
9,0,0-9


In [92]:
edges.to_csv('/Users/Rebeca/Fintech_中信專案/edge.csv')